# Introduction to Data Science 2025

# Week 2

## Exercise 1 | Titanic: data preprocessing and imputation
<span style="font-weight: bold"> *Note: You can find tutorials for NumPy and Pandas under 'Useful tutorials' in the course material.*</span>

Download the [Titanic dataset](https://www.kaggle.com/c/titanic) [train.csv] from Kaggle or <span style="font-weight: 500">directly from the course material</span>, and complete the following exercises. If you choose to download the dataset from Kaggle, you will need to create a Kaggle account unless you already have one, but it is quite straightforward.

The dataset consists of personal information of all the passengers on board the RMS Titanic, along with information about whether they survived the iceberg collision or not.

1. Your first task is to read the data file and print the shape of the data.

    <span style="font-weight: 500"> *Hint 1: You can read them into a Pandas dataframe if you wish.*</span>
    
    <span style="font-weight: 500"> *Hint 2: The shape of the data should be (891, 12).*</span>

In [50]:
import pandas as pd
data = pd.read_csv("train.csv")
shape = data.shape
print(shape)


(891, 12)


2. Let's look at the data and get started with some preprocessing. Some of the columns, e.g <span style="font-weight: 500"> *Name*</span>, simply identify a person and are not useful for prediction tasks. Try to identify these columns, and remove them.

    <span style="font-weight: 500"> *Hint: The shape of the data should now be (891, 9).*</span>

In [51]:
import pandas as pd
df = pd.read_csv('train.csv')
df = df.drop(['Name'], axis=1)
df = df.drop(['Fare'], axis=1)
df = df.drop(['Ticket'], axis=1)
shape = df.shape
print(shape)

(891, 9)


3. The column <span style="font-weight: 500">*Cabin*</span> contains a letter and a number. A smart catch at this point would be to notice that the letter stands for the deck level on the ship. Keeping just the deck information would be more informative when developing, e.g. a classifier that predicts whether a passenger survived. The next step in our preprocessing will be to add a new column to the dataset, which consists simply of the deck letter. You can then remove the original <span style="font-weight: 500">*Cabin*</span>-column.

<span style="font-weight: 500">*Hint: The deck letters should be ['A' 'B' 'C' 'D' 'E' 'F' 'G' 'T'].*</span>

In [52]:
import pandas as pd
df = pd.read_csv('train.csv')
df = df.drop(['Name'], axis=1)
df = df.drop(['Fare'], axis=1)
df = df.drop(['Ticket'], axis=1)
df['Cabin_let'] = df['Cabin']
df['Cabin_let'] = df['Cabin_let'].str.replace('\d+', '', regex=True)
df = df.drop(['Cabin'], axis=1)
df = df.dropna()
shape = df.shape
print(shape)
print(df[['Cabin_let']].to_string(index=False)) 

(183, 9)
Cabin_let
        C
        C
        E
        G
        C
        D
        A
    C C C
        D
        B
        C
        F
      F G
    C C C
        E
        A
      D D
        D
        C
      B B
        E
        D
        D
        C
        B
        F
        C
        B
        A
        C
        F
        F
        B
        B
        G
        A
        D
        D
        C
        C
        C
        D
        G
        C
        B
        E
        B
        C
        C
        C
        D
        B
        D
      C C
      B B
      C C
        C
        E
        C
  B B B B
        C
        E
        C
        D
        B
        C
        C
        C
        E
        T
        F
    C C C
        F
        E
        D
        B
        E
        C
      B B
        D
        G
        C
        E
        C
        E
      B B
    C C C
        A
        C
        C
        C
        E
        E
        E
        D
        B
        C
        B
 

4. You’ll notice that some of the columns, such as the previously added deck number, are [categorical](https://en.wikipedia.org/wiki/Categorical_variable). To preprocess the categorical variables so that they're ready for further computation, we need to avoid the current string format of the values. This means the next step for each categorical variable is to transform the string values to numeric ones, that correspond to a unique integer ID representative of each distinct category. This process is called label encoding and you can read more about it [here](https://pandas.pydata.org/docs/user_guide/categorical.html).

    <span style="font-weight: 500">*Hint: Pandas can do this for you.*</span>

In [53]:
import pandas as pd
df = pd.read_csv('train.csv')
df = df.drop(['Name', 'Fare', 'Ticket'], axis=1)

df['Cabin_let'] = df['Cabin']
df['Cabin_let'] = df['Cabin_let'].str.replace('\d+', '', regex=True)
df = df.drop(['Cabin'], axis=1)

df = df.dropna()

print(df[['Cabin_let']].to_string(index=False))

df["Sex"] = df["Sex"].astype("category").cat.codes
df["Cabin_let"] = df["Cabin_let"].astype("category").cat.codes

print(df.head())

Cabin_let
        C
        C
        E
        G
        C
        D
        A
    C C C
        D
        B
        C
        F
      F G
    C C C
        E
        A
      D D
        D
        C
      B B
        E
        D
        D
        C
        B
        F
        C
        B
        A
        C
        F
        F
        B
        B
        G
        A
        D
        D
        C
        C
        C
        D
        G
        C
        B
        E
        B
        C
        C
        C
        D
        B
        D
      C C
      B B
      C C
        C
        E
        C
  B B B B
        C
        E
        C
        D
        B
        C
        C
        C
        E
        T
        F
    C C C
        F
        E
        D
        B
        E
        C
      B B
        D
        G
        C
        E
        C
        E
      B B
    C C C
        A
        C
        C
        C
        E
        E
        E
        D
        B
        C
        B
        C


5. Next, let's look into missing value **imputation**. Some of the rows in the data have missing values, e.g when the cabin number of a person is unknown. Most machine learning algorithms have trouble with missing values, and they need to be handled during preprocessing:

    a) For continuous variables, replace the missing values with the mean of the non-missing values of that column.

    b) For categorical variables, replace the missing values with the mode of the column.

    <span style="font-weight: 500">*Remember: Even though in the previous step we transformed categorical variables into their numeric representation, they are still categorical.*</span>

In [67]:
import pandas as pd
df = pd.read_csv('train.csv')
df = df.drop(['Name', 'Fare', 'Ticket'], axis=1)

df['Cabin_let'] = df['Cabin']
df['Cabin_let'] = df['Cabin_let'].str.replace('\d+', '', regex=True)
df = df.drop(['Cabin'], axis=1)

for col in df.columns:
    if df[col].dtype in ['float64', 'int64']:
        df[col].fillna(df[col].mean(), inplace=True)
    else:
        df[col].fillna(df[col].mode()[0], inplace=True)


df["Sex"] = df["Sex"].astype("category").cat.codes
df["Cabin_let"] = df["Cabin_let"].astype("category").cat.codes
embarked_map = {"C": 0, "Q": 1, "S": 2}
df["Embarked"] = df["Embarked"].map(embarked_map)


print(df.head())

   PassengerId  Survived  Pclass  Sex   Age  SibSp  Parch  Embarked  Cabin_let
0            1         0       3    1  22.0      1      0         2          5
1            2         1       1    0  38.0      1      0         0          5
2            3         1       3    0  26.0      0      0         2          5
3            4         1       1    0  35.0      1      0         2          5
4            5         0       3    1  35.0      0      0         2          5


6. At this point, all data is numeric. Write the data, with the modifications we made, to a  <span style="font-weight: 500"> .csv</span> file. Then, write another file, this time in <span style="font-weight: 500">JSON</span> format, with the following structure:

In [74]:
#[
#    {
#        "Deck": 0,
#        "Age": 20,
#        "Survived", 0
#        ...
#    },
#    {
#        ...
#    }
#]



In [73]:
import pandas as pd
df = pd.read_csv('train.csv')
df = df.drop(['Name', 'Fare', 'Ticket'], axis=1)

df['Cabin_let'] = df['Cabin']
df['Cabin_let'] = df['Cabin_let'].str.replace('\d+', '', regex=True)
df = df.drop(['Cabin'], axis=1)

for col in df.columns:
    if df[col].dtype in ['float64', 'int64']:
        df[col].fillna(df[col].mean(), inplace=True)
    else:
        df[col].fillna(df[col].mode()[0], inplace=True)


df["Sex"] = df["Sex"].astype("category").cat.codes
df["Cabin_let"] = df["Cabin_let"].astype("category").cat.codes
embarked_map = {"C": 0, "Q": 1, "S": 2}
df["Embarked"] = df["Embarked"].map(embarked_map)


print(df.head())
df.to_csv('train2.csv', index=False)
json_data = df.to_dict(orient="records")
import json
with open("train2.json", "w") as f:
    json.dump(json_data, f, indent=4)


   PassengerId  Survived  Pclass  Sex   Age  SibSp  Parch  Embarked  Cabin_let
0            1         0       3    1  22.0      1      0         2          5
1            2         1       1    0  38.0      1      0         0          5
2            3         1       3    0  26.0      0      0         2          5
3            4         1       1    0  35.0      1      0         2          5
4            5         0       3    1  35.0      0      0         2          5


Study the records and try to see if there is any evident pattern in terms of chances of survival.

**Remember to submit your code on the MOOC platform. You can return this Jupyter notebook (.ipynb) or .py, .R, etc depending on your programming preferences.**

## Exercise 2 | Titanic 2.0: exploratory data analysis

In this exercise, we’ll continue to study the Titanic dataset from the last exercise. Now that we have done some preprocessing, it’s time to look at the data with some exploratory data analysis.

1. First investigate each feature variable in turn. For each categorical variable, find out the mode, i.e., the most frequent value. For numerical variables, calculate the median value.

In [19]:
import pandas as pd
import json

with open("train2.json", "r") as file:
    data = json.load(file)

df = pd.DataFrame(data)

res = {}

for col in df.columns:
    if pd.api.types.is_numeric_dtype(df[col]):
        res[col] = ("median", df[col].median())
    else:
        res[col] = ("mode", df[col].mode().iloc[0])

for col, (stat, value) in res.items():
    print(col, stat, value)


PassengerId median 446.0
Survived median 0.0
Pclass median 3.0
Sex median 1.0
Age median 29.69911764705882
SibSp median 0.0
Parch median 0.0
Embarked median 2.0
Cabin_let median 5.0


2. Next, combine the modes of the categorical variables, and the medians of the numerical variables, to construct an imaginary “average survivor”. This "average survivor" should represent the typical passenger of the class of passengers who survived. Also following the same principle, construct the “average non-survivor”.

    <span style="font-weight: 500">*Hint 1: What are the average/most frequent variable values for a non-survivor?*</span>
    
    <span style="font-weight: 500">*Hint 2: You can split the dataframe in two: one subset containing all the survivors and one consisting of all the non-survivor instances. Then, you can use the summary statistics of each of these dataframe to create a prototype "average survivor" and "average non-survivor", respectively.*</span>

In [22]:
import pandas as pd
import json

with open("train2.json", "r") as file:
	data = json.load(file)

df = pd.DataFrame(data)



def create_res(dataframe):
	res = {}    
	for col in df.columns:
		if col == "Survived":
				continue
		if pd.api.types.is_numeric_dtype(df[col]):
			res[col] = ("median", df[col].median())
		else:
			res[col] = ("mode", df[col].mode().iloc[0])
	return res

df_surv = df[df["Survived"] == 1]
df_nonsurv = df[df["Survived"] == 0]

survivor = create_res(df_surv)
nonsurv = create_res(df_nonsurv)
print(survivor)
print(nonsurv)


{'PassengerId': ('median', 446.0), 'Pclass': ('median', 3.0), 'Sex': ('median', 1.0), 'Age': ('median', 29.69911764705882), 'SibSp': ('median', 0.0), 'Parch': ('median', 0.0), 'Embarked': ('median', 2.0), 'Cabin_let': ('median', 5.0)}
{'PassengerId': ('median', 446.0), 'Pclass': ('median', 3.0), 'Sex': ('median', 1.0), 'Age': ('median', 29.69911764705882), 'SibSp': ('median', 0.0), 'Parch': ('median', 0.0), 'Embarked': ('median', 2.0), 'Cabin_let': ('median', 5.0)}


3. Next, let's study the distributions of the variables in the two groups (survivor/non-survivor). How well do the average cases represent the respective groups? Can you find actual passengers that are very similar to the (average) representative of their own group? Can you find passengers that are very similar to the (average) representative of the other group?

    <span style="font-weight: 500">*Note: Feel free to choose EDA methods according to your preference: non-graphical/graphical, static/interactive - anything goes.*</span>

In [ ]:
import pandas as pd
import json

with open("train2.json", "r") as file:
	data = json.load(file)

df = pd.DataFrame(data)



def create_res(dataframe):
	res = {}    
	for col in df.columns:
		if col == "Survived":
				continue
		if pd.api.types.is_numeric_dtype(df[col]):
			res[col] = ("median", df[col].median())
		else:
			res[col] = ("mode", df[col].mode().iloc[0])
	return res

df_surv = df[df["Survived"] == 1]
df_nonsurv = df[df["Survived"] == 0]

survivor = create_res(df_surv)
nonsurv = create_res(df_nonsurv)



def avg_row(dic):
    avg = {}
	
    for col, (stat, value) in dic.items():
        avg_row[col] = value
    return avg


avg_surv = avg_row(survivor)
avg_nonsurv = avg_row(nonsurv)



4. Next, let's continue the analysis by looking into pairwise and multivariate relationships between the variables in the two groups. Try to visualize two variables at a time using, e.g., scatter plots and use a different color to encode the survival status.

    <span style="font-weight: 500">*Hint 1: You can also check out Seaborn's pairplot function, if you wish.*</span>

    <span style="font-weight: 500">*Hint 2: To better show many data points with the same value for a given variable, you can use either transparency or ‘jitter’.*</span>

In [11]:
# Use this cell for your code

5. Finally, recall the preprocessing we did in the first exercise. What can you say about the effect of the choices that were made to use the mode and mean to impute missing values, instead of, for example, ignoring passengers with missing data?

*This is a good way to preserve all available data, since the dataset is fairly small. It's a good way to widen the amount of data since nothing is lost. If we were to drop the values entirely, the dataset could become unreliable due to biased results. It does reduce the variability though, since the filled in data is already present in the set. This means that some outliers can get less noticable since the entire set is focused more on the average point.*

**Remember to submit your code on the MOOC platform. You can return this Jupyter notebook (.ipynb) or .py, .R, etc depending on your programming preferences.**

## Exercise 3 | Working with text data 2.0

This exercise is related to the second exercise from last week. Find the saved <span style="font-weight: 500">pos.txt</span> and <span style="font-weight: 500">neg.txt</span> files, or, alternatively, you can find the week 1 example solutions on the MOOC platform after Tuesday.

1. Find the most common words in each file (positive and negative). Examine the results. Do they tend to be general terms relating to the nature of the data? How well do they indicate positive/negative sentiment?

In [ ]:
from collections import Counter

with open("pos.txt", "r", encoding="utf-8") as pos_file:
    pos_data = pos_file.read()

split_it = pos_data.split()
Counters_found = Counter(split_it)

most_occur = Counters_found.most_common(10)
print(f"pos {most_occur}")



with open("neg.txt", "r", encoding="utf-8") as neg_file:
    neg_data = neg_file.read()

split_it = neg_data.split()
Counters_found = Counter(split_it)

most_occur = Counters_found.most_common(10)
print(f"neg {most_occur}")

#the words do seem to be more general terms and its not really possible to analyse the sentiment behind them from just the most common words



pos [('great', 435906), ('work', 427780), ('use', 345233), ('good', 291448), ('fit', 273505), ('instal', 229408), ('product', 210084), ('look', 181401), ('just', 180420), ('like', 175725)]
neg [('work', 39998), ('use', 37244), ('fit', 31293), ('product', 24394), ('just', 23580), ('light', 22907), ('like', 20685), ('time', 19665), ('look', 18824), ('instal', 18709)]


2. Compute a [TF/IDF](https://en.wikipedia.org/wiki/Tf–idf) vector for each of the two text files, and make them into a <span style="font-weight: 500">2 x m</span> matrix, where <span style="font-weight: 500">m</span> is the number of unique words in the data. The problem with using the most common words in a review to analyze its contents is that words that are common overall will be common in all reviews (both positive and negative). This means that they probably are not good indicators about the sentiment of a specific review. TF/IDF stands for Term Frequency / Inverse Document Frequency (here the reviews are the documents), and is designed to help by taking into consideration not just the number of times a term occurs (term frequency), but also how many times a word exists in other reviews as well (inverse document frequency). You can use any variant of the formula, as well as off-the-shelf implementations. <span style="font-weight: 500">*Hint: You can use [sklearn](http://scikit-learn.org/).*</span>

In [31]:
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd

with open("pos.txt", "r", encoding="utf-8") as pos_file:
    pos_data = pos_file.read()

with open("neg.txt", "r", encoding="utf-8") as neg_file:
    neg_data = neg_file.read()

vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform([pos_data, neg_data])
tfidf_array = tfidf_matrix.toarray()

print(tfidf_matrix.shape)

words = vectorizer.get_feature_names()

df = pd.DataFrame(tfidf_array, index=["Pos", "Neg"], columns=words)
print(df.head(10))


(2, 235581)
           00       000      0000     00000    000000  00000000  \
Pos  0.000372  0.000072  0.000045  0.000003  0.000002  0.000000   
Neg  0.000219  0.000174  0.000060  0.000008  0.000008  0.000011   

     000000000000000000058  000000000000002  00000000004   0000001  ...  \
Pos               0.000001         0.000001     0.000004  0.000002  ...   
Neg               0.000000         0.000000     0.000000  0.000000  ...   

       zzzzip         zzzzz  zzzzzooooo        zzzzzz   zzzzzzt   zzzzzzz  \
Pos  0.000001  8.414345e-07    0.000001  8.414345e-07  0.000000  0.000000   
Neg  0.000000  1.509088e-05    0.000000  7.545442e-06  0.000011  0.000011   

     zzzzzzzz  zzzzzzzzip  zzzzzzzzzzzzzz  zzzzzzzzzzzzzzzzzzzz  
Pos  0.000001    0.000001        0.000001              0.000002  
Neg  0.000000    0.000000        0.000000              0.000000  

[2 rows x 235581 columns]


3. List the words with the highest TF/IDF score in each class (positive | negative), and compare them to the most common words. What do you notice? Did TF/IDF work as expected?

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd

with open("pos.txt", "r", encoding="utf-8") as pos_file:
    pos_data = pos_file.read()

with open("neg.txt", "r", encoding="utf-8") as neg_file:
    neg_data = neg_file.read()

vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform([pos_data, neg_data])
tfidf_array = tfidf_matrix.toarray()

print(tfidf_matrix.shape)

words = vectorizer.get_feature_names()

df = pd.DataFrame(tfidf_array, index=["Pos", "Neg"], columns=words)
print(df.head(10))

top_pos = df.loc["Pos"].sort_values(ascending=False).head(10)
top_neg = df.loc["Neg"].sort_values(ascending=False).head(10)

print(top_pos)
print(top_neg)


#i notice that they match quite well. I thinki it did work as expected


(2, 235581)
           00       000      0000     00000    000000  00000000  \
Pos  0.000372  0.000072  0.000045  0.000003  0.000002  0.000000   
Neg  0.000219  0.000174  0.000060  0.000008  0.000008  0.000011   

     000000000000000000058  000000000000002  00000000004   0000001  ...  \
Pos               0.000001         0.000001     0.000004  0.000002  ...   
Neg               0.000000         0.000000     0.000000  0.000000  ...   

       zzzzip         zzzzz  zzzzzooooo        zzzzzz   zzzzzzt   zzzzzzz  \
Pos  0.000001  8.414345e-07    0.000001  8.414345e-07  0.000000  0.000000   
Neg  0.000000  1.509088e-05    0.000000  7.545442e-06  0.000011  0.000011   

     zzzzzzzz  zzzzzzzzip  zzzzzzzzzzzzzz  zzzzzzzzzzzzzzzzzzzz  
Pos  0.000001    0.000001        0.000001              0.000002  
Neg  0.000000    0.000000        0.000000              0.000000  

[2 rows x 235581 columns]
great      0.366786
work       0.359949
use        0.290491
good       0.245234
fit        0.230137
ins

4. Plot the words in each class with their corresponding TF/IDF scores. Note that there will be a lot of words, so you’ll have to think carefully to make your chart clear! If you can’t plot them all, plot a subset – think about how you should choose this subset.

    <span style="font-weight: 500">*Hint: you can use word clouds. But feel free to challenge yourselves to think of any other meaningful way to visualize this information!*</span>

In [ ]:
#tried to do this but had no idea on how to proceed

**Remember to submit your code on the MOOC platform. You can return this Jupyter notebook (.ipynb) or .py, .R, etc depending on your programming preferences.**

## Exercise 4 | Junk charts

There’s a thriving community of chart enthusiasts who keep looking for statistical graphics that they find inappropriate, and which they call “junk charts”, and who often also propose ways to improve them.

1. Find at least three statistical visualizations you think are not very good and identify their problems. Copying examples from various junk chart websites is not accepted – you should find your own junk charts, out in the wild. You should be able to find good (or rather, bad) examples quite easily since a significant fraction of charts can have at least *some* issues. The examples you choose should also have different problems, e.g., try to avoid collecting three bar charts, all with problematic axes. Instead, try to find as interesting and diverse examples as you can.

2. Try to produce improved versions of the charts you selected. The data is of course often not available, but perhaps you can try to extract it, at least approximately, from the chart. Or perhaps you can simulate data that looks similar enough to make the point.



**Submit a PDF with all the charts (the ones you found and the ones you produced).**